## SQL实战 | nowcoder 91题总结
### 第四部分 SQL:61-80题

　　ID：wlong
　　数据分析爱好者、目前小白一枚
　　邮箱：wlong826@163.com
　　和鲸社区：[【wlong】](https://www.heywhale.com/home/user/profile/5f0a61d72be199002c073935)
　　如果有不完善的地方，欢迎小伙伴评论区留言！
　　最近博客、和鲸每周都会有所更新，欢迎大家关注点赞分享哦！
 
 ----
 
 　　这里呢，建议SQL不熟练的小伙伴先去**nowcoder网**上刷一遍，遇到不会的时候再过来看**答案**，总共**80道题**左右，但nowcoder网上有些题目存在**断层不连续**，序号写到了91题，所以这里主要也是以官网的序号为准。刷题链接见这[【数据库SQL实战】](https://www.nowcoder.com/ta/sql) (★★★)

　牛客网SQL刷题链接：[【数据库SQL实战】](https://www.nowcoder.com/ta/sql) (★★★)

　如果对爬虫感兴趣的小伙伴，看这看这👉：[【零基础入门Python爬虫--豆瓣、58同城、安居客等】](https://www.heywhale.com/mw/project/60be1fa8acdcb3001707b133)

　关于Pandas数据处理的一些小总结，看这看这👉：[【Pandas数据处理120题知识点总结】](https://www.heywhale.com/mw/project/60c0b06919d601001898394c)

　经典数分案例泰坦尼克号的生存分析，看这看这👉：[【经典案例之泰坦尼克号乘客生存情况预测分析 】](https://www.heywhale.com/mw/project/60d18448056f570017c149dc)
 
 　关于Hive SQL的一点小小总结，看这看这👉：[【《Hive 编程指南》的小小总结】](https://www.heywhale.com/mw/project/60f00c93be5f0900176366a6)

　如果小伙伴对于Python实现数据结构感兴趣，可以看这👉：[【线性表】](https://www.heywhale.com/mw/project/60e7b58cbe5f09001761ac87)、[【栈和队列】](https://www.heywhale.com/mw/project/60ebe5de3f2448001705ccc2)

### 61. 对于employees表中，给出奇数行的first_name(★★★★)

　　对于employees表中，输出first_name排名(按first_name升序排序)为奇数的first_name,输出结果不需要排序；

In [ ]:
select e.first_name
from employees e
join (
        select
        first_name
        ,row_number() over(order by first_name) r
        from employees
    ) b
on e.first_name = b.first_name
where b.r % 2 = 1;

### 62. 出现三次以上相同积分的情况

　　id为用户主键id，number代表积分情况，让你写一个sql查询，积分表里面出现三次以及三次以上的积分；

In [2]:
select number
from grade
group by number
having count(*) >= 3;

### 63. 刷题通过的题目排名

　　输出通过的题目的排名，通过题目个数相同的，排名相同，此时按照id升序排列；

In [ ]:
select 
id
,number
,dense_rank() over(order by number desc) t_rank
from passing_number
order by t_rank, id;

### 64. 找到每个人的任务

　　找到每个人的任务情况，并且输出出来，没有任务的也要输出，而且输出结果按照person的id升序排序；

In [ ]:
select
p.id
,name
,content
from person p
left join task t
on p.id = t.person_id
order by id;

### 65. 异常的邮件概率(★★★★)

　　写一个sql查询，每一个日期里面，正常用户发送给正常用户邮件失败的概率是多少，结果保留到小数点后面3位(3位之后的四舍五入)，并且按照日期升序排序；

In [ ]:
# 方法一：第一次想出来的比较复杂
select
a.date
,round(a.c1/b.c2, 3) p
from
(
	select # 按日期分组，看每一天异常发送数
	date
	,count(*) c1
	from email
	where send_id in (
						select id # 找到正常用户
						from user
						where is_blacklist = 0
					 )
	and receive_id in (

						select id # 找到正常用户
						from user
						where is_blacklist = 0
					  )
	and type = 'no_completed'
	group by date
) a
join
(
	select # 按日期分组，看每一天所有用户的发送数
	date
	,count(*) c2
	from email
	where send_id in (
						select id # 找到正常用户
						from user
						where is_blacklist = 0
					 )
	and receive_id in (

						select id # 找到正常用户
						from user
						where is_blacklist = 0
					  )
	group by date
) b
on a.date = b.date
order by date;

In [ ]:
# 方法二：
select
date
,round(sum(case type when 'no_completed' then 1 else 0 end)/count(*), 3) p # 异常发送数/所有发送数
from email
where send_id in ( # 子查询也可以换成连接查询，对user表进行两次连接查询
					select id # 找到正常用户
					from user
					where is_blacklist = 0
				 )
and receive_id in (
					select id # 找到正常用户
					from user
					where is_blacklist = 0
				  )
group by date
order by date;

### 66. 牛客每个人最近的登录日期(一)

　　写出一个sql语句查询每个用户最近一天登录的日子，并且按照user_id升序排序；

In [3]:
# 方法一：分组使用Max函数
select
user_id
,max(date) d
from login
group by user_id
order by user_id;

In [ ]:
# 方法二：窗口函数
select
user_id
,date d
from (
		select
		user_id
		,date
		,row_number() over(partition by user_id order by date desc) r
		from login
	) a
where a.r = 1
order by 1;

### 67. 牛客每个人最近的登录日期(二)(★★★★)

　　写出一个sql语句查询每个用户最近一天登录的日子，用户的名字，以及用户用的设备的名字，并且查询结果按照user的name升序排序；

In [ ]:
# 方法一：子查询
select
a.name u_n
,c.name c_n
,l.date
from login l
join ( 
		select # 先找到每个用户最近一天登录的用户id,姓名以及日期
		u.id
		,u.name
		,max(date) date
		from login l
		join user u
		on l.user_id = u.id
		group by 1,2
	 ) a
on l.user_id = a.id
and l.date = a.date
join client c
on l.client_id = c.id
order by 1;

In [ ]:
# 方法二：窗口函数
select
u.name u_n
,c.name c_n
,date
from (
		select
		user_id
		,client_id
		,date
		,row_number() over(partition by user_id order by date desc) r
		from login
	) a
join user u
on a.user_id = u.id
join client c
on a.client_id = c.id
where a.r = 1
order by 1;

### 68. 牛客每个人最近的登录日期(三)(★★★★)

　　写出一个sql语句查询新登录用户次日成功的留存率，即第1天登陆之后，第2天再次登陆的概率,保存小数点后面3位(3位之后的四舍五入)；

In [ ]:
# 方法一：
select round(sum(case when a.later_date = date_add(a.date, interval 1 day) then 1 else 0 end) / count(a.user_id), 3) p
from (
		select # 先按用户id分区, 然后将日期列往前一行，添加新的列later_date
		user_id
		,date
		,lead(date,1) over(partition by user_id order by date) later_date
        ,row_number() over(partition by user_id order by date) r
		from login
	 ) a
where r = 1 # 由于题目要求是新用户，所以筛选出每个用户分区后的第一行，然后再select中判断date列和later_date列是否是加一天的关系

In [ ]:
# 方法二：先计算总用户,再计算每个用户的登陆并且第二天也登陆的用户数，再算出留存率即可
select round(count(distinct user_id)*1.0/(select count(distinct user_id) from login) ,3)
from login
where (user_id,date)
in (select user_id,DATE_ADD(min(date),INTERVAL 1 DAY) from login group by user_id);

### 69. 牛客每个人最近的登录日期(四)(★★★★)

　　sql语句查询每个日期登录新用户个数，并且查询结果按照日期升序排序；

In [ ]:
# 方法一：
# 思路：先根据user_id分组，找到每个用户第一次登录的日期; 再根据找到的每个用户第一次登陆的日期进行分组，然后统计用户的个数;
# 这样的话会有一个小问题就是，有的日期没有一个新用户，就会不显示，所以我这里又查询了所有日期，然后进行左连接，最终统计的结果也会包括0了;
select
a.date
,count(b.user_id) new
from (select distinct date from login) a
left join (
		select
		user_id
		,min(date) date
		from login
		group by 1
	 ) b
on a.date = b.date
group by 1
order by 1;

In [ ]:
# 方法二：sum+case(参考牛客大佬)
select distinct date
,sum(case when (user_id, date) in (select user_id, min(date) from login group by 1) then 1 else 0 end)
from login
group by 1
order by 1;

### 70. 牛客每个人最近的登录日期(五) (★★★★★)

　　写出一个sql语句查询每个日期新用户的次日留存率，结果保留小数点后面3位数(3位之后的四舍五入)，并且查询结果按照日期升序排序；

In [ ]:
# 方法一： 刚开始写的代码，感觉自己都被绕晕了，有点过于复杂了些，然后只能在想想其他简便好理解的方法
select 
date
,case when p is null then 0.000 else p end # 为了解决出现的None, 多加了套子查询
from 
(
    select distinct l.date 
    ,round(sum(case when a.later_one_day = date_add(a.date, INTERVAL 1 DAY) then 1 else 0 end)/sum(case when (l.user_id, l.date) in (select user_id, min(date) from login group by 1) then 1 else 0 end), 3) p
    # 上面这步思路就是：根据分区后的列表，找到每个用户第一次登录日期, 下一次登录日期,以及是否是次日登录(1表示是,0表示否)
    from login l
    left join (
                select # 找到这么一个表，用户id, 日期，登录日期升序排列(为了后续找第一次登录日期)，下一次登录日期
                user_id
                ,date
                ,row_number() over(partition by user_id order by date) r # 用于筛选首次登录的日期
                ,lead(date, 1) over(partition by user_id order by date) later_one_day # 下一次登录日期
                from login 
              ) a
    on a.user_id = l.user_id
    where a.r = 1
    group by 1
    order by 1
) b;

In [ ]:
# 方法二：按日期分组，当天首次登陆并且次日又登录的人数/当天登录的总人数
select 
a.date
,round(ifnull(b.first_next_num/a.first_num, 0), 3) p #当天首次登录并且次日也登录的人数/首次登录的人数即为当天留存率
from ( # 新用户表
		select distinct date # 该查询主要目的是：找到每个日期的新用户数
		,sum(case when (user_id, date) in (select user_id, min(date) from login group by 1) then 1 else 0 end) first_num # 首次登录日期总人数
		from login l
		group by 1
	 ) a
left join ( # 次日留存用户表
				select count(user_id) first_next_num # 该查询主要目的是：找到首次登录并且次日又登录的用户数，也就是次日留存用户数
				,date_add(date, interval -1 day) date2 # 首次登录日期
				from login
				where (user_id, date) in ( # 找到首次登录同时次日也登录的用户id和次日登录日期
											select
											user_id
											,date_add(min(date), interval 1 day)
											from login
											group by 1
										)
				group by date2
			) b
on a.date = b.date2
order by a.date;

### 71. 牛客每个人最近的登录日期(六)(★★★★)

　　写出一个sql语句查询刷题信息，包括: 用户的名字，以及截止到某天，累计总共通过了多少题，并且查询结果先按照日期升序排序，再按照姓名升序排序，有登录却没有刷题的哪一天的数据不需要输出；

In [ ]:
select
u.name u_n
,date
,sum(number) over(partition by user_id order by date) ps_num
from passing_number p_n
join user u
on p_n.user_id = u.id
group by 1,2
order by 2,1;

### 72. 考试分数(一)

　　写一个sql语句查询各个岗位分数的平均数，并且按照分数降序排序，结果保留小数点后面3位(3位之后四舍五入)；

In [ ]:
select
job
,round(avg(score),3) avg
from grade
group by job
order by avg desc;

### 73. 考试分数(二)

　　写一个sql语句查询用户分数大于其所在工作(job)分数的平均分的所有grade的属性，并且以id的升序排序；

In [ ]:
select
id
,g.job
,score
from grade g
join (
		select
		job
		,avg(score) avg
		from grade
		group by job
     )	a
on a.job = g.job
where g.score > a.avg
order by id;

### 74. 考试分数(三)(★★★★)

　　找出每个岗位分数排名前2名的用户，得到的结果先按照language的name升序排序，再按照积分降序排序，最后按照grade的id升序排序；

In [ ]:
# 方法一：窗口函数
select
a.id
,l.name
,score
from (
		select
		id
		,language_id
		,score
		,dense_rank() over(partition by language_id order by score desc) r
		from grade
	 ) a
join language l
on a.language_id = l.id
where a.r in (1,2)
order by 2,3 desc,1;

In [ ]:
# 方法二：自连接
select g1.id, l.name, g1.score
from grade g1 join language l on g1.language_id=l.id 
where 
(
    select count(distinct g2.score) 
    from grade g2 
    where g2.score>=g1.score and g1.language_id=g2.language_id
) <=2 order by l.name,g1.score desc ,g1.id;


### 75. 考试分数(四)(★★★★)

　　写一个sql语句查询各个岗位分数升序排列之后的中位数位置的范围，并且按job升序排序；

In [ ]:
# 方法一：case + when
select
job
,case when medium = round(medium, 0) then round(medium,0) else round(medium+0.5,0) end as start # medium = round(medium,0)判断是否medium是小数,进而可以确定count(*)是奇数还是偶数，然后start和end分情况讨论
,case when medium = round(medium, 0) then round(medium+1,0) else round(medium+0.5,0) end as end
from (
		select
		job
		,count(*)/2 medium
		from grade
		group by 1
	 ) a
order by job;

In [ ]:
# 方法二：
select
job
,floor(( count(*) + 1 )/ 2 ) AS start
,floor(( count(*) + 2 )/ 2 ) AS end
from grade
group by job
order by job;

### 76. 考试分数(五)(★★★★★)

　　写一个sql语句查询各个岗位分数的中位数位置上的所有grade信息，并且按id升序排序；

In [ ]:
select
a.id
,a.job
,a.score
,a.t_rank
from ( # 这个子表目的是:按照job分区，对score进行降序，找到不同工作内部的分数排序
		select
		id
		,job
		,score
		,dense_rank() over(partition by job order by score desc) t_rank
		from grade
	 ) a
join ( # 这个子表目的是：找到每个工作中位数的始末位置，然后匹配上一子表的排序后的位置
		select
		job
		,case when medium = round(medium, 0) then round(medium,0) else round(medium+0.5,0) end as start # medium = round(medium,0)判断是否medium是小数,进而可以确定count(*)是奇数还是偶数，然后start和end分情况讨论
		,case when medium = round(medium, 0) then round(medium+1,0) else round(medium+0.5,0) end as end
		from (
				select
				job
				,count(*)/2 medium
				from grade
				group by 1
			 ) e
	 ) b
on a.job = b.job
where a.t_rank in (b.start, b.end) # 判断排序的位置是否是中位数的始末位置
order by id; # 按id升序排列

### 77. 牛客的课程订单分析(一)

　　写出一个sql语句查询在2025-10-15以后状态为购买成功的C++课程或者Java课程或者Python的订单，并且按照order_info的id升序排序；

In [ ]:
select *
from order_info
where date > '2025-10-15'
and status = 'completed'
and product_name in ('C++', 'Java', 'Python')
order by id;

### 78. 牛客的课程订单分析(二)

　　写出一个sql语句查询在2025-10-15以后，同一个用户下单2个以及2个以上状态为购买成功的C++课程或Java课程或Python课程的user_id，并且按照user_id升序排序；

In [ ]:
select user_id
from order_info
where date > '2025-10-15'
and product_name in ('C++', 'Java', 'Python')
and status = 'completed'
group by 1
having count(*) >= 2
order by 1;

### 79. 牛客的课程订单分析(三)

　　写出一个sql语句查询在2025-10-15以后，同一个用户下单2个以及2个以上状态为购买成功的C++课程或Java课程或Python课程的订单信息，并且按照order_info的id升序排序；

In [ ]:
select *
from order_info
where user_id in ( # 先找出来满足条件的用户id
					select user_id
					from order_info
					where date > '2025-10-15'
					and product_name in ('C++', 'Java', 'Python')
					and status = 'completed'
					group by 1
					having count(*) >= 2
				 )
and date > '2025-10-15'
and product_name in ('C++', 'Java', 'Python')
and status = 'completed'
order by id;

### 80. 牛客的课程订单分析(四)(★★★★)

　　写出一个sql语句查询在2025-10-15以后，如果有一个用户下单2个以及2个以上状态为购买成功的C++课程或Java课程或Python课程，那么输出这个用户的user_id，以及满足前面条件的第一次购买成功的C++课程或Java课程或Python课程的日期first_buy_date，以及购买成功的C++课程或Java课程或Python课程的次数cnt，并且输出结果按照user_id升序排序；

In [ ]:
select
user_id
,min(date) first_buy_date
,count(*) cnt
from order_info
where user_id in ( # 先找出来满足条件的用户id
					select user_id
					from order_info
					where date > '2025-10-15'
					and product_name in ('C++', 'Java', 'Python')
					and status = 'completed'
					group by 1
					having count(*) >= 2
				 )
and date > '2025-10-15'
and product_name in ('C++', 'Java', 'Python')
and status = 'completed'
group by 1
order by 1;